In [1]:
#Sourced from https://bojanstavrikj.github.io/content/page1/wunderground_scraper

#some modifications by Joey Hatfield for our use


from bs4 import BeautifulSoup as BS
from selenium import webdriver
from functools import reduce
import pandas as pd
import time

# function to load wunderground data (without this it has no records to show)
def render_page(url):
    driver = webdriver.Chrome('C:/Users/hatfiej1/Downloads/chromedriver.exe')
    driver.get(url)
    time.sleep(3)
    r = driver.page_source
    driver.quit()
    return r


# function to scrape wunderground
def scraper(page, dates):
    output = pd.DataFrame()
    
    for d in dates:
            try:
                url = str(str(page) + str(d))

                r = render_page(url)

                soup = BS(r, "html.parser")
                container = soup.find('lib-city-history-observation')
                if(container.find('tbody')):
                    check = container.find('tbody')
                else:
                    break

                data = []

                for c in check.find_all('tr', class_='ng-star-inserted'):
                    for i in c.find_all('td', class_='ng-star-inserted'):
                        trial = i.text
                        trial = trial.strip('  ')
                        data.append(trial)

                if round(len(data) / 10) >= 1:
                    Time = pd.DataFrame([data[0:len(data)][x:x+1] for x in range(0, len(data), 10)][0:],
                                                columns=['Time'])
                    Temperature = pd.DataFrame([data[1:len(data)][x:x+1] for x in range(0, len(data), 10)][0:],
                                                columns=['Temperature'])
                    Dew_Point = pd.DataFrame([data[2:len(data)][x:x+1] for x in range(0, len(data), 10)][0:],
                                                columns=['Dew Point'])
                    Humidity = pd.DataFrame([data[3:len(data)][x:x+1] for x in range(0, len(data), 10)][0:],
                                                columns=['Humidity'])
                    Wind_Direction = pd.DataFrame([data[4:len(data)][x:x+1] for x in range(0, len(data), 10)][0:],
                                                columns=['Wind Direction'])
                    Wind_Speed = pd.DataFrame([data[5:len(data)][x:x+1] for x in range(0, len(data), 10)][0:],
                                                columns=['Wind Speed'])
                    Wind_Gust = pd.DataFrame([data[6:len(data)][x:x+1] for x in range(0, len(data), 10)][0:],
                                                columns=['Wind Gust'])
                    Pressure = pd.DataFrame([data[7:len(data)][x:x+1] for x in range(0, len(data), 10)][0:],
                                                columns=['Pressure'])
                    Precipitation = pd.DataFrame([data[8:len(data)][x:x+1] for x in range(0, len(data), 10)][0:],
                                                columns=['Precipitation'])
                    Condition = pd.DataFrame([data[9:len(data)][x:x+1] for x in range(0, len(data), 10)][0:],
                                                columns=['Condition'])
                    dfs = [Time, Temperature, Dew_Point, Humidity, Wind_Direction, Wind_Speed, Wind_Gust, Pressure, Precipitation, Condition]

                    df_final = reduce(lambda left, right: pd.merge(left, right, left_index=True, right_index=True), dfs)

                    df_final['Date'] = str(d) + "-" + df_final.iloc[:, :1].astype(str)

                    output = output.append(df_final)

                    print('Scraper done!', str(d) + "-" + df_final.iloc[:, :1].astype(str))

                    output = output[['Date','Time','Temperature',
                             'Dew Point','Humidity',
                             'Wind Direction','Wind Speed',
                             'Wind Gust','Pressure','Precipitation','Condition']]
            except: 
                 print("error occurred, skipping date ", str(d) + "-" + df_final.iloc[0, :1].astype(str))
    return output
        

        

#generate dates (y/m/d) between 1999-1-1 and 2019-12-31
dates = []
year = 1999
for i in range(20):
    for k in range(1,13):
        if k == 1 or k == 3 or k == 5 or k == 7 or k == 8 or k == 10 or k == 12:
            for j in range(1, 32):
                dates.append(str(year + i) + '-' + str(k) + '-' + str(j))
        else:
            if k == 2:
                if ((year + i) - 2000) % 4 == 0:
                    for j in range(1, 30):
                        dates.append(str(year + i) + '-' + str(k) + '-' + str(j))
                else:
                    for j in range(1, 29):
                        dates.append(str(year + i) + '-' + str(k) + '-' + str(j))
            else:
                for j in range(1, 31):
                    dates.append(str(year + i) + '-' + str(k) + '-' + str(j))
page = 'https://www.wunderground.com/history/daily/us/il/rosemont/KORD/date/'

df2 = scraper(page, dates)

#dates

Scraper done!                  Time
0    1999-1-1-1:56 AM
1    1999-1-1-2:56 AM
2    1999-1-1-3:56 AM
3    1999-1-1-4:02 AM
4    1999-1-1-4:56 AM
5    1999-1-1-5:56 AM
6    1999-1-1-6:56 AM
7    1999-1-1-7:56 AM
8    1999-1-1-8:56 AM
9    1999-1-1-9:56 AM
10  1999-1-1-10:56 AM
11  1999-1-1-11:56 AM
12  1999-1-1-12:56 PM
13   1999-1-1-1:56 PM
14   1999-1-1-2:00 PM
15   1999-1-1-2:40 PM
16   1999-1-1-2:56 PM
17   1999-1-1-3:56 PM
18   1999-1-1-4:28 PM
19   1999-1-1-4:56 PM
20   1999-1-1-5:46 PM
21   1999-1-1-5:56 PM
22   1999-1-1-6:56 PM
23   1999-1-1-7:56 PM
24   1999-1-1-8:56 PM
25   1999-1-1-9:56 PM
26  1999-1-1-10:56 PM
27  1999-1-1-11:33 PM
28  1999-1-1-11:56 PM
29  1999-1-1-12:43 AM
Scraper done!                  Time
0   1999-1-2-12:56 AM
1    1999-1-2-1:49 AM
2    1999-1-2-1:56 AM
3    1999-1-2-2:36 AM
4    1999-1-2-2:56 AM
5    1999-1-2-3:56 AM
6    1999-1-2-4:56 AM
7    1999-1-2-5:56 AM
8    1999-1-2-6:56 AM
9    1999-1-2-7:56 AM
10   1999-1-2-8:56 AM
11   1999-1-2-9:56 AM
12  

Scraper done!                   Time
0   1999-1-13-12:56 AM
1    1999-1-13-1:12 AM
2    1999-1-13-1:32 AM
3    1999-1-13-1:56 AM
4    1999-1-13-2:34 AM
5    1999-1-13-2:56 AM
6    1999-1-13-3:15 AM
7    1999-1-13-3:56 AM
8    1999-1-13-4:56 AM
9    1999-1-13-5:56 AM
10   1999-1-13-6:56 AM
11   1999-1-13-7:56 AM
12   1999-1-13-8:45 AM
13   1999-1-13-8:56 AM
14   1999-1-13-9:29 AM
15   1999-1-13-9:46 AM
16   1999-1-13-9:56 AM
17  1999-1-13-10:31 AM
18  1999-1-13-10:48 AM
19  1999-1-13-10:56 AM
20  1999-1-13-11:34 AM
21  1999-1-13-11:43 AM
22  1999-1-13-11:56 AM
23  1999-1-13-12:56 PM
24   1999-1-13-1:56 PM
25   1999-1-13-2:17 PM
26   1999-1-13-2:32 PM
27   1999-1-13-2:56 PM
28   1999-1-13-3:56 PM
29   1999-1-13-4:56 PM
30   1999-1-13-5:56 PM
31   1999-1-13-6:56 PM
32   1999-1-13-7:56 PM
33   1999-1-13-8:56 PM
34   1999-1-13-9:56 PM
35  1999-1-13-10:56 PM
36  1999-1-13-11:56 PM
37  1999-1-13-12:34 AM
Scraper done!                   Time
0   1999-1-14-12:56 AM
1    1999-1-14-1:56 AM
2    1

Scraper done!                   Time
0   1999-1-23-12:56 AM
1    1999-1-23-2:56 AM
2    1999-1-23-3:23 AM
3    1999-1-23-3:56 AM
4    1999-1-23-4:56 AM
5    1999-1-23-5:56 AM
6    1999-1-23-6:07 AM
7    1999-1-23-6:31 AM
8    1999-1-23-6:56 AM
9    1999-1-23-7:18 AM
10   1999-1-23-7:56 AM
11   1999-1-23-8:56 AM
12   1999-1-23-9:56 AM
13  1999-1-23-10:56 AM
14  1999-1-23-11:23 AM
15  1999-1-23-11:56 AM
16  1999-1-23-12:56 PM
17   1999-1-23-1:56 PM
18   1999-1-23-2:37 PM
19   1999-1-23-2:56 PM
20   1999-1-23-3:56 PM
21   1999-1-23-4:26 PM
22   1999-1-23-4:42 PM
23   1999-1-23-4:46 PM
24   1999-1-23-4:56 PM
25   1999-1-23-5:18 PM
26   1999-1-23-5:47 PM
27   1999-1-23-5:56 PM
28   1999-1-23-6:56 PM
29   1999-1-23-7:56 PM
30   1999-1-23-8:56 PM
31   1999-1-23-9:56 PM
32  1999-1-23-10:56 PM
33  1999-1-23-11:56 PM
Scraper done!                   Time
0   1999-1-24-12:56 AM
1    1999-1-24-1:56 AM
2    1999-1-24-2:56 AM
3    1999-1-24-3:56 AM
4    1999-1-24-4:56 AM
5    1999-1-24-5:56 AM
6    1

Scraper done!                  Time
0   1999-2-6-12:56 AM
1    1999-2-6-1:56 AM
2    1999-2-6-2:56 AM
3    1999-2-6-3:56 AM
4    1999-2-6-4:56 AM
5    1999-2-6-5:56 AM
6    1999-2-6-6:56 AM
7    1999-2-6-7:56 AM
8    1999-2-6-9:56 AM
9   1999-2-6-10:56 AM
10  1999-2-6-11:56 AM
11  1999-2-6-12:56 PM
12   1999-2-6-1:56 PM
13   1999-2-6-2:56 PM
14   1999-2-6-3:56 PM
15   1999-2-6-4:56 PM
16   1999-2-6-5:56 PM
17   1999-2-6-6:56 PM
18   1999-2-6-7:56 PM
19   1999-2-6-8:56 PM
20   1999-2-6-9:56 PM
21  1999-2-6-10:56 PM
22  1999-2-6-11:32 PM
23  1999-2-6-11:56 PM
Scraper done!                  Time
0   1999-2-7-12:56 AM
1    1999-2-7-1:56 AM
2    1999-2-7-2:56 AM
3    1999-2-7-3:56 AM
4    1999-2-7-4:56 AM
5    1999-2-7-5:56 AM
6    1999-2-7-6:42 AM
7    1999-2-7-7:56 AM
8    1999-2-7-8:56 AM
9    1999-2-7-9:56 AM
10  1999-2-7-10:56 AM
11  1999-2-7-11:56 AM
12  1999-2-7-12:56 PM
13   1999-2-7-1:56 PM
14   1999-2-7-2:56 PM
15   1999-2-7-3:56 PM
16   1999-2-7-4:56 PM
17   1999-2-7-5:56 PM
18  

Scraper done!                   Time
0   1999-2-19-12:56 AM
1    1999-2-19-1:56 AM
2    1999-2-19-2:56 AM
3    1999-2-19-3:56 AM
4    1999-2-19-4:36 AM
5    1999-2-19-4:56 AM
6    1999-2-19-5:56 AM
7    1999-2-19-6:56 AM
8    1999-2-19-7:56 AM
9    1999-2-19-8:56 AM
10   1999-2-19-9:56 AM
11  1999-2-19-10:26 AM
12  1999-2-19-10:56 AM
13  1999-2-19-11:04 AM
14  1999-2-19-11:48 AM
15  1999-2-19-11:56 AM
16  1999-2-19-12:30 PM
17  1999-2-19-12:37 PM
18  1999-2-19-12:56 PM
19   1999-2-19-1:56 PM
20   1999-2-19-2:56 PM
21   1999-2-19-3:56 PM
22   1999-2-19-4:49 PM
23   1999-2-19-4:56 PM
24   1999-2-19-5:56 PM
25   1999-2-19-6:56 PM
26   1999-2-19-7:56 PM
27   1999-2-19-8:56 PM
28   1999-2-19-9:56 PM
29  1999-2-19-10:56 PM
30  1999-2-19-11:56 PM
Scraper done!                   Time
0   1999-2-20-12:56 AM
1    1999-2-20-2:56 AM
2    1999-2-20-3:56 AM
3    1999-2-20-4:56 AM
4    1999-2-20-5:56 AM
5    1999-2-20-6:56 AM
6    1999-2-20-7:56 AM
7    1999-2-20-8:56 AM
8    1999-2-20-9:56 AM
9   19

...

Scraper done!                    Time
0   1999-12-22-12:56 AM
1    1999-12-22-1:56 AM
2    1999-12-22-2:56 AM
3    1999-12-22-3:56 AM
4    1999-12-22-4:56 AM
5    1999-12-22-5:56 AM
6    1999-12-22-6:56 AM
7    1999-12-22-7:56 AM
8    1999-12-22-8:56 AM
9    1999-12-22-9:56 AM
10  1999-12-22-10:56 AM
11  1999-12-22-11:56 AM
12  1999-12-22-12:56 PM
13   1999-12-22-1:56 PM
14   1999-12-22-2:56 PM
15   1999-12-22-3:56 PM
16   1999-12-22-4:56 PM
17   1999-12-22-5:56 PM
18   1999-12-22-6:56 PM
19   1999-12-22-7:56 PM
20   1999-12-22-8:56 PM
21   1999-12-22-9:56 PM
22  1999-12-22-10:56 PM
23  1999-12-22-11:56 PM
Scraper done!                    Time
0   1999-12-23-12:56 AM
1    1999-12-23-1:56 AM
2    1999-12-23-2:56 AM
3    1999-12-23-3:56 AM
4    1999-12-23-4:56 AM
5    1999-12-23-5:56 AM
6    1999-12-23-6:56 AM
7    1999-12-23-7:56 AM
8    1999-12-23-8:56 AM
9    1999-12-23-9:56 AM
10  1999-12-23-10:56 AM
11  1999-12-23-11:56 AM
12  1999-12-23-12:56 PM
13   1999-12-23-1:56 PM
14   1999-12

Scraper done!                  Time
0   2000-1-4-12:56 AM
1    2000-1-4-1:56 AM
2    2000-1-4-2:56 AM
3    2000-1-4-3:56 AM
4    2000-1-4-4:56 AM
5    2000-1-4-5:03 AM
6    2000-1-4-5:56 AM
7    2000-1-4-6:56 AM
8    2000-1-4-7:56 AM
9    2000-1-4-8:56 AM
10   2000-1-4-9:56 AM
11  2000-1-4-10:56 AM
12  2000-1-4-11:56 AM
13  2000-1-4-12:56 PM
14   2000-1-4-1:56 PM
15   2000-1-4-3:56 PM
16   2000-1-4-4:56 PM
17   2000-1-4-5:56 PM
18   2000-1-4-6:56 PM
19   2000-1-4-7:23 PM
20   2000-1-4-7:56 PM
21   2000-1-4-8:56 PM
22   2000-1-4-9:56 PM
23  2000-1-4-10:56 PM
24  2000-1-4-11:56 PM
Scraper done!                  Time
0   2000-1-5-12:56 AM
1    2000-1-5-1:56 AM
2    2000-1-5-2:56 AM
3    2000-1-5-3:56 AM
4    2000-1-5-4:17 AM
5    2000-1-5-4:56 AM
6    2000-1-5-5:56 AM
7    2000-1-5-6:56 AM
8    2000-1-5-7:56 AM
9    2000-1-5-8:56 AM
10   2000-1-5-9:56 AM
11  2000-1-5-10:56 AM
12  2000-1-5-11:56 AM
13  2000-1-5-12:56 PM
14   2000-1-5-1:56 PM
15   2000-1-5-2:56 PM
16   2000-1-5-3:56 PM
17  

Scraper done!                   Time
0   2000-1-17-12:56 AM
1    2000-1-17-1:56 AM
2    2000-1-17-2:56 AM
3    2000-1-17-3:56 AM
4    2000-1-17-4:56 AM
5    2000-1-17-5:56 AM
6    2000-1-17-6:56 AM
7    2000-1-17-7:56 AM
8    2000-1-17-8:56 AM
9    2000-1-17-9:56 AM
10  2000-1-17-10:56 AM
11  2000-1-17-11:56 AM
12  2000-1-17-12:56 PM
13   2000-1-17-1:56 PM
14   2000-1-17-2:04 PM
15   2000-1-17-2:06 PM
16   2000-1-17-2:11 PM
17   2000-1-17-2:18 PM
18   2000-1-17-2:22 PM
19   2000-1-17-2:31 PM
20   2000-1-17-2:42 PM
21   2000-1-17-2:56 PM
22   2000-1-17-3:15 PM
23   2000-1-17-3:22 PM
24   2000-1-17-3:30 PM
25   2000-1-17-3:56 PM
26   2000-1-17-4:12 PM
27   2000-1-17-4:23 PM
28   2000-1-17-4:41 PM
29   2000-1-17-4:56 PM
30   2000-1-17-4:58 PM
31   2000-1-17-5:02 PM
32   2000-1-17-5:25 PM
33   2000-1-17-5:36 PM
34   2000-1-17-5:56 PM
35   2000-1-17-6:30 PM
36   2000-1-17-6:56 PM
37   2000-1-17-7:56 PM
38   2000-1-17-8:30 PM
39   2000-1-17-8:56 PM
40   2000-1-17-9:56 PM
41  2000-1-17-10:56 

Scraper done!                   Time
0   2000-1-30-12:56 AM
1    2000-1-30-1:23 AM
2    2000-1-30-1:56 AM
3    2000-1-30-2:17 AM
4    2000-1-30-2:56 AM
5    2000-1-30-3:56 AM
6    2000-1-30-4:18 AM
7    2000-1-30-4:32 AM
8    2000-1-30-4:56 AM
9    2000-1-30-5:56 AM
10   2000-1-30-6:56 AM
11   2000-1-30-7:38 AM
12   2000-1-30-7:56 AM
13   2000-1-30-8:56 AM
14   2000-1-30-9:56 AM
15  2000-1-30-10:25 AM
16  2000-1-30-10:31 AM
17  2000-1-30-10:39 AM
18  2000-1-30-10:56 AM
19  2000-1-30-11:56 AM
20  2000-1-30-12:56 PM
21   2000-1-30-1:56 PM
22   2000-1-30-2:56 PM
23   2000-1-30-3:56 PM
24   2000-1-30-4:56 PM
25   2000-1-30-5:05 PM
26   2000-1-30-5:41 PM
27   2000-1-30-5:56 PM
28   2000-1-30-6:56 PM
29   2000-1-30-7:49 PM
30   2000-1-30-7:56 PM
31   2000-1-30-8:27 PM
32   2000-1-30-8:40 PM
33   2000-1-30-8:56 PM
34   2000-1-30-9:26 PM
35   2000-1-30-9:56 PM
36  2000-1-30-10:46 PM
37  2000-1-30-10:56 PM
38  2000-1-30-11:13 PM
39  2000-1-30-11:56 PM
Scraper done!                   Time
0   20

Scraper done!                   Time
0   2000-2-12-12:56 AM
1    2000-2-12-1:56 AM
2    2000-2-12-2:56 AM
3    2000-2-12-3:56 AM
4    2000-2-12-4:56 AM
5    2000-2-12-5:56 AM
6    2000-2-12-6:56 AM
7    2000-2-12-7:56 AM
8    2000-2-12-8:56 AM
9    2000-2-12-9:56 AM
10  2000-2-12-10:56 AM
11  2000-2-12-11:56 AM
12  2000-2-12-12:56 PM
13   2000-2-12-1:56 PM
14   2000-2-12-3:56 PM
15   2000-2-12-4:56 PM
16   2000-2-12-5:56 PM
17   2000-2-12-6:56 PM
18   2000-2-12-7:56 PM
19   2000-2-12-8:56 PM
20   2000-2-12-9:56 PM
21  2000-2-12-10:56 PM
22  2000-2-12-11:56 PM
Scraper done!                   Time
0   2000-2-13-12:56 AM
1    2000-2-13-1:56 AM
2    2000-2-13-2:56 AM
3    2000-2-13-3:56 AM
4    2000-2-13-4:56 AM
5    2000-2-13-5:56 AM
6    2000-2-13-6:56 AM
7    2000-2-13-7:56 AM
8    2000-2-13-8:39 AM
9    2000-2-13-8:56 AM
10   2000-2-13-9:17 AM
11   2000-2-13-9:30 AM
12   2000-2-13-9:39 AM
13   2000-2-13-9:56 AM
14  2000-2-13-10:56 AM
15  2000-2-13-11:56 AM
16  2000-2-13-12:56 PM
17   2

# len(dates)

In [18]:
dates[0]

'1999-1-1'

In [3]:
df2.to_csv('./wunderground_scraped_data_daily_partial_1999.csv',index=False)

In [3]:
df

,Date,Time,Temperature,Dew Point,Humidity,Wind Direction,Wind Speed,Wind Gust,Pressure,Precipitation,Condition
0,1999-1-1-1:56 AM,1:56 AM,12 °F,5 °F,73 °%,NW,9 °mph,0 °mph,29.56 °in,0.0 °in,Mostly Cloudy
1,1999-1-1-2:56 AM,2:56 AM,13 °F,7 °F,77 °%,N,9 °mph,0 °mph,29.59 °in,0.0 °in,Cloudy
2,1999-1-1-3:56 AM,3:56 AM,9 °F,0 °F,67 °%,N,13 °mph,0 °mph,29.63 °in,0.0 °in,Mostly Cloudy
3,1999-1-1-4:02 AM,4:02 AM,9 °F,1 °F,73 °%,N,10 °mph,0 °mph,29.59 °in,0.0 °in,Partly Cloudy
4,1999-1-1-4:56 AM,4:56 AM,7 °F,-2 °F,67 °%,N,8 °mph,0 °mph,29.65 °in,0.0 °in,Partly Cloudy
...,...,...,...,...,...,...,...,...,...,...,...
23,2000-1-25-7:56 PM,7:56 PM,15 °F,5 °F,64 °%,NNW,14 °mph,23 °mph,29.44 °in,0.0 °in,Partly Cloudy
24,2000-1-25-8:56 PM,8:56 PM,14 °F,2 °F,59 °%,NNW,13 °mph,0 °mph,29.45 °in,0.0 °in,Fair
25,2000-1-25-9:56 PM,9:56 PM,14 °F,3 °F,62 °%,NW,15 °mph,0 °mph,29.44 °in,0.0 °in,Fair
26,2000-1-25-10:56 PM,10:56 PM,14 °F,5 °F,67 °%,NNW,16 °mph,0 °mph,29.44 °in,0.0 °in,Partly Cloudy


In [2]:
df2

,Date,Time,Temperature,Dew Point,Humidity,Wind Direction,Wind Speed,Wind Gust,Pressure,Precipitation,Condition
0,1999-1-1-1:56 AM,1:56 AM,12 °F,5 °F,73 °%,NW,9 °mph,0 °mph,29.56 °in,0.0 °in,Mostly Cloudy
1,1999-1-1-2:56 AM,2:56 AM,13 °F,7 °F,77 °%,N,9 °mph,0 °mph,29.59 °in,0.0 °in,Cloudy
2,1999-1-1-3:56 AM,3:56 AM,9 °F,0 °F,67 °%,N,13 °mph,0 °mph,29.63 °in,0.0 °in,Mostly Cloudy
3,1999-1-1-4:02 AM,4:02 AM,9 °F,1 °F,73 °%,N,10 °mph,0 °mph,29.59 °in,0.0 °in,Partly Cloudy
4,1999-1-1-4:56 AM,4:56 AM,7 °F,-2 °F,67 °%,N,8 °mph,0 °mph,29.65 °in,0.0 °in,Partly Cloudy
...,...,...,...,...,...,...,...,...,...,...,...
6,2000-2-22-6:56 AM,6:56 AM,41 °F,31 °F,67 °%,S,6 °mph,0 °mph,29.39 °in,0.0 °in,Cloudy
7,2000-2-22-7:56 AM,7:56 AM,39 °F,0 °F,76 °%,SSE,7 °mph,0 °mph,29.39 °in,0.0 °in,Cloudy
8,2000-2-22-8:56 AM,8:56 AM,42 °F,38 °F,85 °%,S,8 °mph,0 °mph,29.39 °in,0.0 °in,Cloudy
9,2000-2-22-9:56 AM,9:56 AM,44 °F,38 °F,78 °%,S,8 °mph,0 °mph,29.39 °in,0.0 °in,Mostly Cloudy


In [14]:
dates

['2000-1-1',
 '2000-1-2',
 '2000-1-3',
 '2000-1-4',
 '2000-1-5',
 '2000-1-6',
 '2000-1-7',
 '2000-1-8',
 '2000-1-9',
 '2000-1-10',
 '2000-1-11',
 '2000-1-12',
 '2000-1-13',
 '2000-1-14',
 '2000-1-15',
 '2000-1-16',
 '2000-1-17',
 '2000-1-18',
 '2000-1-19',
 '2000-1-20',
 '2000-1-21',
 '2000-1-22',
 '2000-1-23',
 '2000-1-24',
 '2000-1-25',
 '2000-1-26',
 '2000-1-27',
 '2000-1-28',
 '2000-1-29',
 '2000-1-30',
 '2000-1-31',
 '2000-2-1',
 '2000-2-2',
 '2000-2-3',
 '2000-2-4',
 '2000-2-5',
 '2000-2-6',
 '2000-2-7',
 '2000-2-8',
 '2000-2-9',
 '2000-2-10',
 '2000-2-11',
 '2000-2-12',
 '2000-2-13',
 '2000-2-14',
 '2000-2-15',
 '2000-2-16',
 '2000-2-17',
 '2000-2-18',
 '2000-2-19',
 '2000-2-20',
 '2000-2-21',
 '2000-2-22',
 '2000-2-23',
 '2000-2-24',
 '2000-2-25',
 '2000-2-26',
 '2000-2-27',
 '2000-2-28',
 '2000-2-29',
 '2000-3-1',
 '2000-3-2',
 '2000-3-3',
 '2000-3-4',
 '2000-3-5',
 '2000-3-6',
 '2000-3-7',
 '2000-3-8',
 '2000-3-9',
 '2000-3-10',
 '2000-3-11',
 '2000-3-12',
 '2000-3-13',
 '200